In [1]:
import os
import glob
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import time

In [2]:
def count_files_and_dirs(path):
    # Get all directories in the specified path
    dir_count = len([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])

    # Get all files in the specified path and its subdirectories
    file_count = len(glob.glob(path + '**/*', recursive=True)) - dir_count

    return dir_count, file_count

def get_files(path):
    path_dir = glob.glob(path + '**/*.jpg', recursive=True)

    return path_dir
dir_count, file_count = count_files_and_dirs('data/DN8~06_27_02/')
img_dir = get_files('data/DN8~06_27_02/')
print("Number of directories:", dir_count)
print("Number of files:", file_count)

img_dir[:3]

Number of directories: 0
Number of files: 77


['data/DN8~06_27_02\\F001-0001.jpg',
 'data/DN8~06_27_02\\F001-0002.jpg',
 'data/DN8~06_27_02\\F001-0003.jpg']

In [ ]:
# Initialize lists to store image width and height
img_widths = []
img_heights = []

for img_path in img_dir:
    img = Image.open(img_path)
    width, height = img.size
    img_widths.append(width)
    img_heights.append(height)

# Plotting
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(img_widths, bins=20, color='blue', edgecolor='black')
plt.title('Image Widths')
plt.xlabel('Width')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(img_heights, bins=20, color='blue', edgecolor='black')
plt.title('Image Heights')
plt.xlabel('Height')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [3]:
len(img_dir)

77

In [12]:
from tqdm import tqdm

def compute_latency_ms_pytorch(model, input_size, iterations=None, device=None):
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

    model.eval()
    # model = model.cpu()
    # input = torch.randn(*input_size)
    model = model.cuda()
    input = torch.randn(*input_size).cuda()

    with torch.no_grad():
        if iterations is None:
            elapsed_time = 0
            iterations = 100
            while elapsed_time < 1:
                torch.cuda.synchronize()
                torch.cuda.synchronize()
                t_start = time.time()
                for _ in range(iterations):
                    model(input)
                torch.cuda.synchronize()
                torch.cuda.synchronize()
                elapsed_time = time.time() - t_start
                iterations *= 2
            FPS = iterations / elapsed_time
            iterations = int(FPS * 6)

        print('=========Speed Testing=========')
        torch.cuda.synchronize()
        torch.cuda.synchronize()
        model(input)
        t_start = time.time()
        for _ in tqdm(range(iterations)):
            model(input)
        torch.cuda.synchronize()
        torch.cuda.synchronize()
        elapsed_time = time.time() - t_start    # μs ()
        latency = elapsed_time / iterations * 1000
    torch.cuda.empty_cache()
    # FPS = 1000 / latency (in ms)
    return latency

In [ ]:
import numpy as np
import random
# Load a pretrained DeepLabv3 model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('pytorch/vision:v0.9.0', 'deeplabv3_resnet101', pretrained=True)
model.to(device)
model.eval()

latency_list = []
fps_list = []

# 492 658
# 512 672
# Load an image
inputDimension = (1, 3, 512, 672)

numbers = list(range(1, 12346))  # 1부터 12345까지의 숫자 리스트 생성
random_numbers = random.sample(numbers, 77)  # 리스트에서 10개의 무작위 숫자 추출

for seed in random_numbers:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    latency = compute_latency_ms_pytorch(model, inputDimension, iterations=1)
    latency_list.append(latency)
    fps_list.append((1000./latency))

    methodName = 'DeepLabv3+'
    print("{} FPS:{:.2f}".format(methodName, (1000./latency)))
    print("{} Latency:{:.2f}ms / {:.4f}s".format(methodName, latency, (latency/1000.)))

avg_latency = np.average(latency_list)
std_latency = np.std(latency_list)
avg_fps = np.average(fps_list)
std_fps = np.std(fps_list)

In [26]:
print(avg_latency)
print(std_latency)
print(avg_fps)
print(std_fps)
len(latency_list)

99.92103783083168
1.9597177354868087
10.011849941273365
0.2014243771636844


2310

In [22]:
print(avg_latency)
print(std_latency)
print(avg_fps)
print(std_fps)

99.22541271556507
1.9728992563723315
10.082169947142576
0.20660599523355105


In [11]:
std_fps = np.std(fps_list)
print(avg_latency)
print(std_latency)
print(avg_fps)
print(std_fps)

54.46811940246847
0.35450331551744835
18.36014026829725
0.11936039713668381
